# LMM usage example

Below we will fit a linear mixed model, and demostrate many inference and prediction methods available for objects of class `LMM`.

## Example data

The data set, which is simulated, contains two numeric variables *Age* and *Aggression*, and two categorical variables *Location* and *Species*. These data are available for 100 (human and alien) individuals.

We will fit the model with the method `LMM#from_formula`, which mimics the behaviour of the function `lmer` from the `R` package `lme4`.

The data is supplied to `LMM#from_formula` as a `Daru::DataFrame` (from the excellent Ruby gem [daru](https://github.com/v0dro/daru.git)). We load the data, and display the first 10 lines with:

In [1]:
require 'daru'
alien_species = Daru::DataFrame.from_csv '../examples/data/alien_species.csv'
alien_species.head

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"http://d3js.org/d3.v3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    window.dispatchEvent(event);\n\t

,Age,Aggression,Location,Species
0,204.95,877.54242028595,Asylum,Dalek
1,39.88,852.528392188206,OodSphere,WeepingAngel
2,107.34,388.791416909388,Asylum,Human
3,210.01,170.010124622982,OodSphere,Ood
4,270.22,1078.31219494376,OodSphere,Dalek
5,157.65,164.924992952256,OodSphere,Ood
6,136.15,865.838374677443,OodSphere,WeepingAngel
7,241.31,1052.36035549029,Earth,Dalek
8,86.84,-8.57251993382567,Asylum,Ood
9,206.7,1070.71900405899,OodSphere,Dalek


## Linear mixed model

We model the *Aggression* level of an individual as a linear function of the *Age* (*Aggression* decreases with *Age*), with a different constant added for each *Species* (i.e. each species has a different base level of aggression). Moreover, we assume that there is a random fluctuation in *Aggression* due to the *Location* that an individual is at. Additionally, there is a random fluctuation in how *Age* affects *Aggression* at each different *Location*. 

Thus, the *Aggression* level of an individual of *Species* $spcs$ who is at the *Location* $lctn$ can be expressed as:
$$Aggression = \beta_{0} + \gamma_{spcs} + Age \cdot \beta_{1} + b_{lctn,0} + Age \cdot b_{lctn,1} + \epsilon,$$
where $\epsilon$ is a random residual, and the random vector $(b_{lctn,0}, b_{lctn,1})^T$ follows a multivariate normal distribution (the same distribution but different realizations of the random vector for each *Location*). That is, we have a linear mixed model with fixed effects $\beta_{0}, \beta_{1}, \gamma_{Dalek}, \gamma_{Ood}, \dots$, and random effects $b_{Asylum,0}, b_{Asylum,1}, b_{Earth,0},\dots$.

We fit this model in `mixed_models` using a syntax familiar from the `R` package `lme4`, and display the estimated fixed and random effects coefficients:

In [2]:
require 'mixed_models'
model_fit = LMM.from_formula(formula: "Aggression ~ Age + Species + (Age | Location)", 
                             data: alien_species)
puts "Fixed effects:"
puts model_fit.fix_ef
puts "Random effects:"
puts model_fit.ran_ef

Fixed effects:
{:intercept=>1016.2867207696772, :Age=>-0.06531615343467667, :Species_lvl_Human=>-499.693695290209, :Species_lvl_Ood=>-899.5693213535769, :Species_lvl_WeepingAngel=>-199.58895804200762}
Random effects:
{:intercept_Asylum=>-116.68080682806675, :Age_Asylum=>-0.03353391213062394, :intercept_Earth=>83.86571630094429, :Age_Earth=>-0.13613996644462312, :intercept_OodSphere=>32.81508992422805, :Age_OodSphere=>0.16967387859838903}


## Some model attributes

Apart from the fixed and random effects coefficients, we can access many attributes of the fitted model. Among others:

* `reml` is an indicator whether the profiled REML criterion or the profiled deviance function was optimized by the model fitting algorithm.

* `formula` returns the R-like formula used to fit the model as a String.

* `model_data`, `optimization_result` and `dev_fun` store the various model matrices in an `LMMData` object, the results of the utilized optimization algorithm, and the corresponding objective function as a `Proc`.

* `sigma2` is the residual variance (unless `weights` was specified in the model fit).

* `sigma_mat` is the covariance matrix of the multivariate normal random effects vector.

* `fix_ef_names` and `ran_ef_names` are Arrays of names of the fixed and random effects.

We can look at some of these parameters for our example model:

In [4]:
puts "REML criterion used: \t#{model_fit.reml}"
puts "Residual variance: \t#{model_fit.sigma2}"
puts "Formula: \t" + model_fit.formula
puts "Variance of the intercept due to 'location' (i.e. variance of b0): \t#{model_fit.sigma_mat[0,0]}"
puts "Variance of the effect of 'age' due to 'location' (i.e. variance of b1): \t#{model_fit.sigma_mat[1,1]}"
puts "Covariance of b0 and b1: \t#{model_fit.sigma_mat[0,1]}"

REML criterion used: 	true
Residual variance: 	0.9496833447256808
Formula: 	Aggression ~ Age + Species + (Age | Location)
Variance of the intercept due to 'location' (i.e. variance of b0): 	10870.932406181153
Variance of the effect of 'age' due to 'location' (i.e. variance of b1): 	0.024233390356817052
Covariance of b0 and b1: 	-0.9716403033290782


Some further convenience methods are:

* `sigma` returns the square root of `sigma2`.

* `theta` returns the optimal solution of the minimization of the deviance function or the REML criterion (whichever was used to fit the model).

* `deviance` returns  the value of the deviance function or the REML criterion at the optimal solution.


In [22]:
puts "Residual standard deviation: \t#{model_fit.sigma}"
puts "REML criterion: \t#{model_fit.deviance}"

Residual standard deviation: 	0.9745169802141371
REML criterion: 	333.715539101513


## Fitted values and residual

There are methods to get the fitted values of the response variable, with or without inclusion of random effects, as well as the model residuals.

In [5]:
puts "Fitted values at the population level:"
model_fit.fitted(with_ran_ef: false)

Fitted values at the population level:


[1002.9001751232403, 814.0929545286947, 509.58198956979004, 103.00035403328388, 998.6369897885589, 106.42030782712357, 807.8049684375385, 1000.5252797843553, 111.04534465183303, 1002.7858718547295, 1008.0797460906101, 498.79959896079356, 100.90435866956511, 111.03358774421474, 102.37593160644838, 812.6239942379489, 97.21856813124623, 800.9297901270043, 510.0032787594437, 502.69505435163774, 503.7329280297147, 999.8864878037642, 497.2875300087809, 1003.3325680589778, 804.1328942914408, 811.9113950039764, 99.91285946042672, 803.9356395080681, 998.4671677896288, 99.50267401685687, 500.71140277182656, 1014.3853675431938, 112.41241174322079, 1010.3031079535265, 114.52212349916078, 813.8107887458568, 811.7526767511301, 498.69901208450415, 1005.0836941325615, 110.67826786953015, 501.0020596546109, 810.6305452351226, 511.31809292808373, 1000.7623774213233, 508.8465296821156, 1008.3730156195318, 1011.249539016795, 102.45953628284474, 1010.9490847109954, 1007.0346876356552, 506.2286582524538, 50

In [21]:
puts "Model residuals:"
model_fit.residuals

Model residuals:


[-1.8041727180520866, -1.1462465432205136, -0.5102357042341055, -1.4385305789776055, 1.010839756116411, -1.0594917601316638, 2.1172177445057514, 0.8212947077422541, -0.024972828168568384, 0.046451573745571295, -0.25215128087575067, 0.5337610637693615, -0.530629854101111, -0.874628273213915, -0.5173146938876982, -1.0767035113831298, -0.37352823345418074, 0.43428801048560217, -0.008686417241960953, 0.9633162152579189, -1.8591988675071889, -0.06035456182212329, -0.8747549066539477, 2.156663624529642, -0.12014582672793495, 1.5012618544112684, 0.5916114379899646, -1.4808531218287726, -0.16478112799109113, 2.0314537895965294, 0.010773844424193157, 0.8539551421492888, -1.2568149547066971, 0.07403493882111434, -0.11003692957027056, 1.0271335859753208, 1.289389380032162, -0.1484680679627104, -0.4342951733153768, 1.7157974749034395, 1.0173666823257577, -0.6984875406659512, 0.16845508494225214, -1.4232161610595995, -0.14893911781359748, 0.1337897922234106, -0.2102464993108697, 0.4063849522508747,

## Fixed effects hypotheses tests and confidence intervals

Often statistical models are used in order to determine which of the predictor variables have a significant relationship with the response variable. `LMM` has a number of methods to aid with this kind of statistical inference.

### Variances and covariances of the fixed effects coefficient estimates

The covariance matrix of the fixed effects estimates is returned by `LMM#fix_ef_cov_mat`, and the standard deviations of the fixed effects coefficients are returned by `LMM#fix_ef_sd`. Methods for hypotheses tests and confidence intervals can be based on these values.

In [6]:
model_fit.fix_ef_sd

{:intercept=>60.19727495932258, :Age=>0.08988486367253856, :Species_lvl_Human=>0.2682523406941927, :Species_lvl_Ood=>0.2814470814004366, :Species_lvl_WeepingAngel=>0.2757835779525997}

### Wald tests and confidence intervals

The [Wald Z test statistics](https://en.wikipedia.org/wiki/Wald_test#Test_on_a_single_parameter) for the fixed effects coefficients can be computed with:

In [7]:
model_fit.fix_ef_z

{:intercept=>16.882603431075875, :Age=>-0.7266646548258817, :Species_lvl_Human=>-1862.7747813759402, :Species_lvl_Ood=>-3196.2289922406044, :Species_lvl_WeepingAngel=>-723.7158917283754}

Based on the above Wald Z test statistics, we can carry out hypotheses tests for each fixed effects terms $\beta_{i}$ or $\gamma_{species}$, testing the null $H_{0} : \beta_{i} = 0$ against the alternative $H_{a} : \beta_{i} \neq 0$, or respectively the null $H_{0} : \gamma_{species} = 0$ against the alternative $H_{a} : \gamma_{species} \neq 0$.

The corresponding (approximate) p-values are obtained with:

In [12]:
model_fit.fix_ef_p(method: :wald)

{:intercept=>0.0, :Age=>0.4674314106158888, :Species_lvl_Human=>0.0, :Species_lvl_Ood=>0.0, :Species_lvl_WeepingAngel=>0.0}

We see that the aggression level of each species is significantly different from the base level (which is the species `Dalek` in this model), while statistically we don't have enough evidence to conclude that the age of an individual is a good predictor of his/her/its aggression level.

We can use the Wald method for confidence intervals as well. For example 90% confidence intervals for each fixed effects coefficient estimate can be computed as follows.

In [8]:
model_fit.fix_ef_conf_int(level: 0.9, method: :wald)

{:intercept=>[917.2710135369496, 1115.302428002405], :Age=>[-0.2131635992213468, 0.08253129235199347], :Species_lvl_Human=>[-500.13493113101106, -499.25245944940696], :Species_lvl_Ood=>[-900.0322606117458, -899.1063820954081], :Species_lvl_WeepingAngel=>[-200.04258166587766, -199.13533441813757]}

## Predictions and prediction intervals

One might also fit a statistical model in order to predict future observations based on new data input.

We consider the following new data set containing age, geographic location and species for ten individuals.

In [9]:
newdata = Daru::DataFrame.from_csv '../examples/data/alien_species_newdata.csv'

,Age,Location,Species
0,209,OodSphere,Dalek
1,90,Earth,Ood
2,173,Asylum,Ood
3,153,Asylum,Human
4,255,OodSphere,WeepingAngel
5,256,Asylum,WeepingAngel
6,37,Earth,Dalek
7,146,Earth,WeepingAngel
8,127,Asylum,WeepingAngel
9,41,Asylum,Ood


Based on the fitted linear mixed model we can predict the aggression levels for the inidividuals, where we can specify whether the random effects estimates should be included in the calculations or not.

In [10]:
puts "Predictions of aggression levels on a new data set:"
puts model_fit.predict(newdata: newdata, with_ran_ef: true)

Predictions of aggression levels on a new data set:
[1070.9125752531213, 182.45206492790766, -17.064468754763425, 384.78815861991046, 876.1240725686444, 674.711339114886, 1092.6985606350875, 871.150885526236, 687.4629975728096, -4.0162601001437395]


Since the estimated fixed and random effects coefficients most likely are not exactly the true values, we probably should look at interval estimates of the predictions, rather than the point estimates produced above.

Two types of such interval estimates are currently available in `LMM`. On the one hand, a *confidence interval* is an interval estimate of the mean value of the response for given covariates (i.e. a population parameter); on the other hand, a *prediction interval* is an interval estimate of a future observation (for further explanation of this distinction see for example <https://stat.ethz.ch/education/semesters/ss2010/seminar/06_Handout.pdf>).

In [11]:
puts "88% confidence intervals for the predictions:"
ci = model_fit.predict_with_intervals(newdata: newdata, level: 0.88, type: :confidence)
Daru::DataFrame.new(ci, order: [:pred, :lower88, :upper88])

88% confidence intervals for the predictions:


,pred,lower88,upper88
0,1002.6356447018298,906.2754736806305,1098.9958157230292
1,110.83894560697945,17.153931194406766,204.52396001955213
2,105.41770487190126,10.164688001453413,200.67072174234912
3,506.59965400396266,411.8519192433839,601.3473887645414
4,800.0421436018271,701.9091175621678,898.1751696414864
5,799.9768274483924,701.8009453651559,898.152709531629
6,1013.8700230925942,920.443931383714,1107.2961148014745
7,807.1616043262068,712.5717592728961,901.7514493795175
8,808.4026112414656,714.1916401880408,902.6135822948904
9,114.0394371252786,20.614034935160902,207.4648393153963


In [12]:
puts "88% prediction intervals for the predictions:"
pi = model_fit.predict_with_intervals(newdata: newdata, level: 0.88, type: :prediction)
Daru::DataFrame.new(pi, order: [:pred, :lower88, :upper88])

88% prediction intervals for the predictions:


,pred,lower88,upper88
0,1002.6356447018298,809.9100502106367,1195.361239193023
1,110.83894560697945,-76.53615878138834,298.21404999534724
2,105.41770487190126,-85.09352857986573,295.92893832366826
3,506.59965400396266,317.0988996180352,696.1004083898902
4,800.0421436018271,603.7713981525592,996.312889051095
5,799.9768274483924,603.6203777718086,996.3332771249762
6,1013.8700230925942,827.0127232975983,1200.7273228875902
7,807.1616043262068,617.9767304767107,996.3464781757028
8,808.4026112414656,619.975479314019,996.8297431689122
9,114.0394371252786,-72.81614465010145,300.89501890065867
